In [ ]:
import numpy as np
from PIL import Image

In [ ]:
dt = np.load(fr"F:\datasets\LG_3D_DataBase\DataSave\Save_L\6554\3D.npy")

In [ ]:
mask=Image.open(fr"F:\datasets\LG_3D_DataBase\DataSave\Save_L\6554\MASK.png")

In [ ]:
mask=np.asarray(mask)

In [ ]:
mask_area=np.count_nonzero(mask)

In [ ]:
da_data = dt[mask>0]
np.mean(da_data)

In [ ]:
up = da_data[da_data>44317]
up_c = np.count_nonzero(up)
up_c/mask_area *100

In [ ]:
up

In [ ]:
from scipy.spatial import Delaunay
import numpy as np

# 生成点集
points = np.random.rand(30, 2)


In [ ]:
tri = Delaunay(points, qhull_options='Qbb')

# 打印剖分结果
print(tri.simplices)

In [ ]:
import numpy as np
from scipy.spatial import Delaunay
import open3d as o3d

# 生成三维点集
points = np.random.rand(30, 3)  # 30个随机点

# 执行 Delaunay 三角剖分
tri = Delaunay(points)

# 查看 Delaunay 输出的 simplex 数组
print("Simplices shape:", tri.simplices.shape)
print("Simplices dtype:", tri.simplices.dtype)

# 将面索引转换为 int32 类型
triangles = np.asarray(tri.simplices).astype(np.int32)

# 创建 Open3D 网格
mesh = o3d.geometry.TriangleMesh()

# 设置顶点
mesh.vertices = o3d.utility.Vector3dVector(points)

# 设置面
mesh.triangles = o3d.utility.Vector3iVector(triangles)

# 计算法线
mesh.compute_vertex_normals()

# 可视化网格
o3d.visualization.draw_geometries([mesh])


In [ ]:
import numpy as np
from scipy.spatial import Delaunay
import open3d as o3d

# 生成三维点集
points = np.random.rand(30, 3)  # 30个随机点

# 执行 Delaunay 三角剖分
tri = Delaunay(points)

# 获取三角形面索引并检查形状
triangles = np.asarray(tri.simplices)
print("triangles shape:", triangles.shape)  # 输出应为 (num_faces, 3)

# 确保 triangles 数据类型为 int32
triangles = triangles.astype(np.int32)
print("triangles dtype:", triangles.dtype)  # 应该是 int32

# 创建 Open3D 网格
mesh = o3d.geometry.TriangleMesh()

# 设置顶点
mesh.vertices = o3d.utility.Vector3dVector(points)

# 设置面
mesh.triangles = o3d.utility.Vector3iVector(triangles)

# 计算法线
mesh.compute_vertex_normals()

# 可视化网格
o3d.visualization.draw_geometries([mesh])


In [27]:
import numpy as np
import open3d as o3d
from scipy.spatial import Delaunay

# 生成环形点集
def generate_ring_points(radius, n_points, z):
    # 均匀分布在 0 到 2*pi 之间
    angles = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    points = np.array([(radius * np.cos(angle), radius * np.sin(angle), z) for angle in angles])
    return points

# 设置环形的半径和点数
radius = 1.0  # 半径
n_points = 50  # 点的数量

# 生成环形点集
points = np.vstack([generate_ring_points(1.0, 50, 2), generate_ring_points(1.1, 50, 1)])

# 使用 Delaunay 三角剖分
tri = Delaunay(points[:, :2])  # 仅使用二维坐标进行 Delaunay 三角剖分

# 获取三角形索引
triangles = np.asarray(tri.simplices)

# 计算每个三角形的法线
def compute_normal(v0, v1, v2):
    # 计算两个边
    edge1 = v1 - v0
    edge2 = v2 - v0
    # 计算法线
    normal = np.cross(edge1, edge2)
    normal = normal / np.linalg.norm(normal)  # 归一化
    return normal

# 筛选有效的三角形
valid_triangles = []

for triangle in triangles:
    # 获取当前三角形的顶点
    v0, v1, v2 = points[triangle]
    
    # 计算三角形的法线
    normal = compute_normal(v0, v1, v2)
    
    # 判断法线的z分量，如果是朝下的法线（即z分量为负），则是底部的面

    if normal[2] < 1:  # 只保留法线朝上的面
        valid_triangles.append(triangle)

# 更新有效的三角形列表
valid_triangles = np.array(valid_triangles)

# 创建 Open3D 网格对象
mesh = o3d.geometry.TriangleMesh()

# 设置顶点和三角形
mesh.vertices = o3d.utility.Vector3dVector(points)
mesh.triangles = o3d.utility.Vector3iVector(valid_triangles)

# 计算法线
mesh.compute_vertex_normals()

# 可视化结果
o3d.visualization.draw_geometries([mesh])


In [28]:
import numpy as np
import open3d as o3d
from scipy.spatial import Delaunay

# 生成环形点集的函数
def generate_ring_points(radius, n_points, z):
    angles = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    points = np.array([(radius * np.cos(angle), radius * np.sin(angle), z) for angle in angles])
    return points

# 生成两个环形网格
points1 = generate_ring_points(1.0, 50, 2)  # 半径 1.0, 50 个点
points2 = generate_ring_points(2.0, 50, 1)  # 半径 2.0, 50 个点

# 使用 Delaunay 三角剖分生成三角形网格
tri1 = Delaunay(points1[:, :2])
tri2 = Delaunay(points2[:, :2])

# 获取三角形索引
triangles1 = tri1.simplices
triangles2 = tri2.simplices

# 合并顶点
all_points = np.vstack([points1, points2])  # 合并顶点

# 更新第二组三角形的索引（因为第二组顶点是从第一个网格之后的，所以需要加上第一个网格的顶点数）
triangles2 += len(points1)

# 合并三角形索引
all_triangles = np.vstack([triangles1, triangles2])

# 创建 Open3D 网格对象
mesh = o3d.geometry.TriangleMesh()

# 设置顶点和三角形
mesh.vertices = o3d.utility.Vector3dVector(all_points)
mesh.triangles = o3d.utility.Vector3iVector(all_triangles)

# 计算法线
mesh.compute_vertex_normals()

# 可视化结果
o3d.visualization.draw_geometries([mesh])


In [30]:
import numpy as np
import open3d as o3d
from scipy.spatial import Delaunay

# 生成环形点集的函数
def generate_ring_points(radius, n_points, z):
    angles = np.linspace(0, 2 * np.pi, n_points, endpoint=False)
    points = np.array([(radius * np.cos(angle), radius * np.sin(angle), z) for angle in angles])
    return points

# 生成两个环形网格
n_points = 50  # 点的数量
points1 = generate_ring_points(1.0, n_points, 2)  # 半径 1.0, z = 2
points2 = generate_ring_points(2.0, n_points, 1)  # 半径 2.0, z = 1

# 使用 Delaunay 三角剖分生成三角形网格
tri1 = Delaunay(points1[:, :2])
tri2 = Delaunay(points2[:, :2])

# 获取三角形索引
triangles1 = tri1.simplices
triangles2 = tri2.simplices

# 合并顶点
all_points = np.vstack([points1, points2])  # 合并顶点

# 更新第二组三角形的索引（因为第二组顶点是从第一个网格之后的，所以需要加上第一个网格的顶点数）
triangles2 += len(points1)

# 合并三角形索引
all_triangles = np.vstack([triangles1, triangles2])

# 创建 Open3D 网格对象
mesh = o3d.geometry.TriangleMesh()

# 设置顶点和三角形
mesh.vertices = o3d.utility.Vector3dVector(all_points)
mesh.triangles = o3d.utility.Vector3iVector(all_triangles)

# 计算法线
mesh.compute_vertex_normals()

# 可视化结果
o3d.visualization.draw_geometries([mesh])
